In [1]:
import torch
import torch.nn as nn
import torch.onnx
import numpy as np
import onnxruntime as ort

In [3]:
# 1. モデル作成
class SimpleModel(nn.Module):
    def __init__(self):
        super(SimpleModel, self).__init__()
        self.fc = nn.Linear(4, 3)  # 入力4次元、出力3次元

    def forward(self, x):
        return self.fc(x)

model = SimpleModel()
model.eval()  # 推論モードに設定

# ダミーデータで確認
dummy_input = torch.randn(1, 4)  # バッチサイズ1, 入力次元4
print("PyTorch推論結果:", model(dummy_input))

# 2. ONNX形式に変換
onnx_path = "simple_model.onnx"
torch.onnx.export(
    model,
    dummy_input,
    onnx_path,
    export_params=True,
    opset_version=11,
    input_names=["input"],
    output_names=["output"]
)
print(f"ONNX形式で保存しました: {onnx_path}")

# 3. ONNXで推論
# ONNX Runtimeセッションを作成
ort_session = ort.InferenceSession(onnx_path)

# 推論データ準備
onnx_input = dummy_input.numpy()
onnx_outputs = ort_session.run(
    None, {"input": onnx_input}
)

print("ONNX推論結果:", onnx_outputs[0])


PyTorch推論結果: tensor([[ 1.0414, -0.8350, -0.3696]], grad_fn=<AddmmBackward0>)
ONNX形式で保存しました: simple_model.onnx
ONNX推論結果: [[ 1.0413516  -0.83499575 -0.36956984]]
